In [1]:
import pandas as pd
import numpy as np
import requests
import string
from bs4 import BeautifulSoup
from multiprocessing import Pool

In [290]:
pd.set_option('max_columns', 999)

In [6]:
#import list of unique fight urls
fight_urls = list(pd.read_csv('fight_urls.csv')['fight_url'])

In [5]:
#write function for getting the data from one url
def getFightData(url):
    fight_url = 'http://www.fightmetric.com/fight-details/{}'.format(url)
    

['b33e6a1d4911cf87',
 'dae788f34bcfcab9',
 '77a8a6e2c3a09d19',
 'cdfc9e763372599a',
 'fd163ed5c9776080',
 'be307e6647a655c8',
 '499557c33e67deaf',
 'b2cda211e0e72811',
 '63f744d672fbb8b4',
 '5724a9cf463c8390',
 'a2c89a8d4857906a',
 '5511eaf7873c4a21',
 'ca78df0ac32551f2',
 '72656e864ca1c972',
 'd5778494ce663d7f',
 'ebfe035b70080668',
 '32d350a70ce39d8f',
 'fd5832aa1ca446bc',
 '43682a646b9b956f',
 'dab5e3680cf6eac7',
 '1a0bf11582ae518b',
 'd8c7497043e7e5bd',
 '99a7172fb7438244',
 'f0e0effb1b2de6ae',
 '60c62fb7bd17a8b3',
 '0e9b7e682e0a79f1',
 'f85bbc760000f001',
 '4a1e7c670be02cb1',
 'fb57dcc651894ae9',
 '038c472ae3fd7728',
 '107079f548338ac3',
 'a730a20ab0c61499',
 'e74e3c2a96fa9c55',
 'f7e8e689a1e8ac12',
 '06287b822837a616',
 '91a3854320fa2cec',
 'd98d5a3800cd21fb',
 'ac82d28aaa5faaf0',
 '383f0de18e7ab9c5',
 '9cba1f0cdf10a8e5',
 '205ad947e8c94901',
 'a5f39e8ff1398269',
 'c8998aa536614a5a',
 '6954b6e02651e353',
 '2af50cc7f34f6fc2',
 '955f171f7facad27',
 'e7cb40d82d987b9e',
 '8935fc5943a

In [145]:
url = 'c46d51fb87a4f485'

In [146]:
fight_url = 'http://www.fightmetric.com/fight-details/{}'.format(url)

response = requests.get(fight_url)
soup = BeautifulSoup(response.content, 'lxml')

In [248]:
summary = soup.find('div', {'class':'b-fight-details__fight'})

In [149]:
elements = summary.find_all('i')[3:]


In [155]:
elements[13].text.split('\n')[-2].strip()

u'27 - 29.'

In [164]:
totals, sig_strikes = soup.find_all('table', {'class': 'b-fight-details__table js-fight-table'})

In [231]:
top = 'fighter_a'
bottom = 'fighter_b'
rounds = sig_strikes.find('tbody').find_all('tr')
Round = rounds[0]
round_stats= Round.find_all('td')

In [246]:
round_stats[8].text.split('\n')[top_ind].strip()

u'9 of 17'

In [188]:
round_stats= rounds[0].find_all('td')

In [339]:
overall_stats = soup.find_all('section', {'class':'b-fight-details__section js-fight-section'})[1].find_all('tr')[1].find_all('td')

In [356]:
print overall_stats[5].text.split('\n')[top_ind].strip().split(' ')[0]
print overall_stats[2].text.split('\n')[top_ind].strip().split(' ')[-1]
print overall_stats[2].text.split('\n')[bottom_ind].strip().split(' ')[0]
print overall_stats[2].text.split('\n')[bottom_ind].strip().split(' ')[-1]

4
232
93
232


In [191]:
len(round_stats)

10

In [274]:
def resultStats(result):
    fight_dict = {}
    fighters = result.find_all('div', {'class': 'b-fight-details__person'})
    fight_dict['fighter_a'] = fighters[0].find('a')['href'].split('/')[-1]
    fight_dict['fighter_b'] = fighters[1].find('a')['href'].split('/')[-1]
    fight_dict['fighter_a_result'] = fighters[0].find('i').text.strip()
    fight_dict['fighter_b_result'] = fighters[1].find('i').text.strip()
    return fight_dict

In [387]:
def entireFightStats(soup, fight_dict):
    overall_stats = soup.find_all('section', {'class':'b-fight-details__section js-fight-section'})[1].find_all('tr')[1].find_all('td')
    top = 'fighter_a'
    bottom = 'fighter_b'
    top_ind = 2
    bottom_ind = 5
    fight_dict['total' + '_' + top + '_'+ 'kd'] = overall_stats[1].text.split('\n')[top_ind].strip()
    fight_dict['total' + '_' + bottom + '_'+ 'kd'] = overall_stats[1].text.split('\n')[bottom_ind].strip()
    fight_dict['total' + '_' + top + '_'+ 'sub_att'] = overall_stats[7].text.split('\n')[top_ind].strip()
    fight_dict['total' + '_' + bottom + '_'+ 'sub_att'] = overall_stats[7].text.split('\n')[bottom_ind].strip()
    fight_dict['total' + '_' + top + '_'+ 'passes'] = overall_stats[8].text.split('\n')[top_ind].strip()
    fight_dict['total' + '_' + bottom + '_'+ 'passes'] = overall_stats[8].text.split('\n')[bottom_ind].strip()
    fight_dict['total' + '_' + top + '_'+ 'reverses'] = overall_stats[9].text.split('\n')[top_ind].strip()
    fight_dict['total' + '_' + bottom + '_'+ 'reverses'] = overall_stats[9].text.split('\n')[bottom_ind].strip()
    fight_dict['total' + '_' + top + '_'+ 'ssl'] = overall_stats[2].text.split('\n')[top_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + top + '_'+ 'ssa'] = overall_stats[2].text.split('\n')[top_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + bottom + '_'+ 'ssl']  = overall_stats[2].text.split('\n')[bottom_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + bottom + '_'+ 'ssa']  = overall_stats[2].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + top + '_'+ 'tsl'] = overall_stats[4].text.split('\n')[top_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + top + '_'+ 'tsa'] = overall_stats[4].text.split('\n')[top_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + bottom + '_'+ 'tsl']  = overall_stats[4].text.split('\n')[bottom_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + bottom + '_'+ 'tsa']  = overall_stats[4].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + top + '_'+ 'tdl'] = overall_stats[5].text.split('\n')[top_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + top + '_'+ 'tda'] = overall_stats[5].text.split('\n')[top_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + bottom + '_'+ 'tdl']  = overall_stats[5].text.split('\n')[bottom_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + bottom + '_'+ 'tda']  = overall_stats[5].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    return fight_dict

In [388]:
sig_stats = soup.find_all('tbody', {'class': 'b-fight-details__table-body'})[2].find('tr').find_all('td')

In [389]:
def entireSigStats(soup, fight_dict):
    top = 'fighter_a'
    bottom = 'fighter_b'
    top_ind = 2
    bottom_ind = 5
    fight_dict['total' + '_' + top + '_'+ 'hsl'] = sig_stats[3].text.split('\n')[top_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + top + '_'+ 'hsa'] = sig_stats[3].text.split('\n')[top_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + bottom + '_'+ 'hsl'] = sig_stats[3].text.split('\n')[bottom_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + bottom + '_'+ 'hsa'] = sig_stats[3].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + top + '_'+ 'bsl'] = sig_stats[4].text.split('\n')[top_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + top + '_'+ 'bsa'] = sig_stats[4].text.split('\n')[top_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + bottom + '_'+ 'bsl'] = sig_stats[4].text.split('\n')[bottom_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + bottom + '_'+ 'bsa'] = sig_stats[4].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + top + '_'+ 'lsl'] = sig_stats[5].text.split('\n')[top_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + top + '_'+ 'lsa'] = sig_stats[5].text.split('\n')[top_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + bottom + '_'+ 'lsl'] = sig_stats[5].text.split('\n')[bottom_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + bottom + '_'+ 'lsa'] = sig_stats[5].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + top + '_'+ 'dsl'] = sig_stats[6].text.split('\n')[top_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + top + '_'+ 'dsa'] = sig_stats[6].text.split('\n')[top_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + bottom + '_'+ 'dsl'] = sig_stats[6].text.split('\n')[bottom_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + bottom + '_'+ 'dsa'] = sig_stats[6].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + top + '_'+ 'csl'] = sig_stats[7].text.split('\n')[top_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + top + '_'+ 'csa'] = sig_stats[7].text.split('\n')[top_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + bottom + '_'+ 'csl'] = sig_stats[7].text.split('\n')[bottom_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + bottom + '_'+ 'csa'] = sig_stats[7].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + top + '_'+ 'gsl'] = sig_stats[8].text.split('\n')[top_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + top + '_'+ 'gsa'] = sig_stats[8].text.split('\n')[top_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + bottom + '_'+ 'gsl'] = sig_stats[8].text.split('\n')[bottom_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + bottom + '_'+ 'gsa'] = sig_stats[8].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    return fight_dict

In [390]:
def summaryStats(summary, fight_dict):
    fight_dict['weight_class'] = summary.find('i').text.strip()
    elements = summary.find_all('i')[3:]
    fight_dict['method'] = elements[0].text.strip()
    fight_dict['round'] = elements[1].text.strip()[-1]
    fight_dict['time'] = elements[3].text.strip().split(' ')[-1]
    fight_dict['num_rounds'] = elements[5].text.split('Rnd')[0][-2]
    fight_dict['referee'] = elements[7].text.split('\n')[-3].strip()
    if elements[11]=='\n \n':
        pass
    else:
        fight_dict['score1'] = elements[11].text.split('\n')[-2].strip()
        fight_dict['score2'] = elements[12].text.split('\n')[-2].strip()
        fight_dict['score3'] = elements[13].text.split('\n')[-2].strip()
    return fight_dict

In [391]:
def totalStats(totals, fight_dict):
    top = 'fighter_a'
    bottom = 'fighter_b'
    rounds = totals.find('tbody').find_all('tr')
    top_ind = 2
    bottom_ind = 5
    for i, Round in enumerate(rounds):
        r = str(i + 1)
        round_stats = Round.find_all('td')
        fight_dict[r+ '_'+ top + '_'+ 'kd'] = round_stats[1].text.split('\n')[top_ind].strip()
        fight_dict[r+ '_'+ bottom + '_'+ 'kd'] = round_stats[1].text.split('\n')[bottom_ind].strip()
        fight_dict[r+ '_'+ top + '_'+ 'ssl'] = round_stats[2].text.split('\n')[top_ind].strip().split(' ')[0]
        fight_dict[r+ '_'+ top + '_'+ 'ssa'] = round_stats[2].text.split('\n')[top_ind].strip().split(' ')[-1]
        fight_dict[r+ '_'+ bottom + '_'+ 'ssl'] = round_stats[2].text.split('\n')[bottom_ind].strip().split(' ')[0]
        fight_dict[r+ '_'+ bottom + '_'+ 'ssa'] = round_stats[2].text.split('\n')[bottom_ind].strip().split(' ')[-1]
        fight_dict[r+ '_'+ top + '_'+ 'tsl'] = round_stats[4].text.split('\n')[top_ind].strip().split(' ')[0]
        fight_dict[r+ '_'+ top + '_'+ 'tsa'] = round_stats[4].text.split('\n')[top_ind].strip().split(' ')[-1]
        fight_dict[r+ '_'+ bottom + '_'+ 'tsl'] = round_stats[4].text.split('\n')[bottom_ind].strip().split(' ')[0]
        fight_dict[r+ '_'+ bottom + '_'+ 'tsa'] = round_stats[4].text.split('\n')[bottom_ind].strip().split(' ')[-1]
        fight_dict[r+ '_'+ top + '_'+ 'tdl'] = round_stats[5].text.split('\n')[top_ind].strip().split(' ')[0]
        fight_dict[r+ '_'+ top + '_'+ 'tda'] = round_stats[5].text.split('\n')[top_ind].strip().split(' ')[-1]
        fight_dict[r+ '_'+ bottom + '_'+ 'tdl'] = round_stats[5].text.split('\n')[bottom_ind].strip().split(' ')[0]
        fight_dict[r+ '_'+ bottom + '_'+ 'tda'] = round_stats[5].text.split('\n')[bottom_ind].strip().split(' ')[-1]
        fight_dict[r+ '_'+ top + '_'+ 'sub_att'] = round_stats[7].text.split('\n')[top_ind].strip()
        fight_dict[r+ '_'+ bottom + '_'+ 'sub_att'] = round_stats[7].text.split('\n')[bottom_ind].strip()
        fight_dict[r+ '_'+ top + '_'+ 'passes'] = round_stats[8].text.split('\n')[top_ind].strip()
        fight_dict[r+ '_'+ bottom + '_'+ 'passes'] = round_stats[8].text.split('\n')[bottom_ind].strip()
        fight_dict[r+ '_'+ top + '_'+ 'reverses'] = round_stats[9].text.split('\n')[top_ind].strip()
        fight_dict[r+ '_'+ bottom + '_'+ 'reverses'] = round_stats[9].text.split('\n')[bottom_ind].strip()
    return fight_dict

In [392]:
def significantStrikes(sig_strikes, fight_dict):
    top = 'fighter_a'
    bottom = 'fighter_b'
    rounds = sig_strikes.find('tbody').find_all('tr')
    top_ind = 2
    bottom_ind = 5
    for i, Round in enumerate(rounds):
        r = str(i+1)
        round_stats = Round.find_all('td')
        fight_dict[r + '_' + top + '_' + 'hsl'] = round_stats[3].text.split('\n')[top_ind].strip().split(' ')[0]
        fight_dict[r + '_' + top + '_' + 'hsa'] = round_stats[3].text.split('\n')[top_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + bottom + '_' + 'hsl'] = round_stats[3].text.split('\n')[bottom_ind].strip().split(' ')[0]
        fight_dict[r + '_' + bottom + '_' + 'hsa'] = round_stats[3].text.split('\n')[bottom_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + top + '_' + 'bsl'] = round_stats[4].text.split('\n')[top_ind].strip().split(' ')[0]
        fight_dict[r + '_' + top + '_' + 'bsa'] = round_stats[4].text.split('\n')[top_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + bottom + '_' + 'bsl'] = round_stats[4].text.split('\n')[bottom_ind].strip().split(' ')[0]
        fight_dict[r + '_' + bottom + '_' + 'bsa'] = round_stats[4].text.split('\n')[bottom_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + top + '_' + 'lsl'] = round_stats[5].text.split('\n')[top_ind].strip().split(' ')[0]
        fight_dict[r + '_' + top + '_' + 'lsa'] = round_stats[5].text.split('\n')[top_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + bottom + '_' + 'lsl'] = round_stats[5].text.split('\n')[bottom_ind].strip().split(' ')[0]
        fight_dict[r + '_' + bottom + '_' + 'lsa'] = round_stats[5].text.split('\n')[bottom_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + top + '_' + 'dsl'] = round_stats[6].text.split('\n')[top_ind].strip().split(' ')[0]
        fight_dict[r + '_' + top + '_' + 'dsa'] = round_stats[6].text.split('\n')[top_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + bottom + '_' + 'dsl'] = round_stats[6].text.split('\n')[bottom_ind].strip().split(' ')[0]
        fight_dict[r + '_' + bottom + '_' + 'dsa'] = round_stats[6].text.split('\n')[bottom_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + top + '_' + 'csl'] = round_stats[7].text.split('\n')[top_ind].strip().split(' ')[0]
        fight_dict[r + '_' + top + '_' + 'csa'] = round_stats[7].text.split('\n')[top_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + bottom + '_' + 'csl'] = round_stats[7].text.split('\n')[bottom_ind].strip().split(' ')[0]
        fight_dict[r + '_' + bottom + '_' + 'csa'] = round_stats[7].text.split('\n')[bottom_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + top + '_' + 'gsl'] = round_stats[8].text.split('\n')[top_ind].strip().split(' ')[0]
        fight_dict[r + '_' + top + '_' + 'gsa'] = round_stats[8].text.split('\n')[top_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + bottom + '_' + 'gsl'] = round_stats[8].text.split('\n')[bottom_ind].strip().split(' ')[0]
        fight_dict[r + '_' + bottom + '_' + 'gsa'] = round_stats[8].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    return fight_dict

        

In [408]:
def Driver(url):
    #takes in a fight url and returns a dictionary of ALL the stats from the fight.
    #this includes round by round stats as well as overall totals
    fight_url = 'http://www.fightmetric.com/fight-details/{}'.format(url)
    response = requests.get(fight_url)
    soup = BeautifulSoup(response.content, 'lxml')
    result = soup.find('div', {'class': 'b-fight-details__persons clearfix'})
    fight_dict = resultStats(result)
    fight_dict = entireFightStats(soup, fight_dict)
    fight_dict = entireSigStats(soup, fight_dict)
    summary = soup.find('div', {'class':'b-fight-details__fight'})
    fight_dict = summaryStats(summary, fight_dict)
    totals, sig_strikes = soup.find_all('table', {'class': 'b-fight-details__table js-fight-table'})
    fight_dict = totalStats(totals, fight_dict)
    fight_dict = significantStrikes(sig_strikes, fight_dict)
    return fight_dict

In [409]:
d  = Driver(url)

In [410]:
url

'c46d51fb87a4f485'

In [396]:
len(fight_urls)

6622

In [411]:
#check to see if the script is written correctly
df = pd.DataFrame([d])

cols_a = []
cols_b = []
for key in d.keys():
    if 'fighter_a' in key:
        cols_a.append(key)
    elif 'fighter_b' in key:
        cols_b.append(key)

unifiedA = {x: x.replace('_a_', '_') for x in cols_a }

unifiedB = {x: x.replace('_b_', '_') for x in cols_b }

A = df[[x for x in df.columns if 'fighter_a' in x]]

B = df[[x for x in df.columns if 'fighter_b' in x]]

A.rename_axis(unifiedA, axis=1, inplace=True)

B.rename_axis(unifiedB, axis=1, inplace=True)

data = pd.concat([A,B])

/Users/adelsona/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:20: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
/Users/adelsona/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:22: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead


In [412]:
data.head()

,1_fighter_bsa,1_fighter_bsl,1_fighter_csa,1_fighter_csl,1_fighter_dsa,1_fighter_dsl,1_fighter_gsa,1_fighter_gsl,1_fighter_hsa,1_fighter_hsl,1_fighter_kd,1_fighter_lsa,1_fighter_lsl,1_fighter_passes,1_fighter_reverses,1_fighter_ssa,1_fighter_ssl,1_fighter_sub_att,1_fighter_tda,1_fighter_tdl,1_fighter_tsa,1_fighter_tsl,2_fighter_bsa,2_fighter_bsl,2_fighter_csa,2_fighter_csl,2_fighter_dsa,2_fighter_dsl,2_fighter_gsa,2_fighter_gsl,2_fighter_hsa,2_fighter_hsl,2_fighter_kd,2_fighter_lsa,2_fighter_lsl,2_fighter_passes,2_fighter_reverses,2_fighter_ssa,2_fighter_ssl,2_fighter_sub_att,2_fighter_tda,2_fighter_tdl,2_fighter_tsa,2_fighter_tsl,3_fighter_bsa,3_fighter_bsl,3_fighter_csa,3_fighter_csl,3_fighter_dsa,3_fighter_dsl,3_fighter_gsa,3_fighter_gsl,3_fighter_hsa,3_fighter_hsl,3_fighter_kd,3_fighter_lsa,3_fighter_lsl,3_fighter_passes,3_fighter_reverses,3_fighter_ssa,3_fighter_ssl,3_fighter_sub_att,3_fighter_tda,3_fighter_tdl,3_fighter_tsa,3_fighter_tsl,fighter_a,fighter_b,fighter_result,total_fighter_bsa,total_fighter_bsl,total_fighter_csa,total_fighter_csl,total_fighter_dsa,total_fighter_dsl,total_fighter_gsa,total_fighter_gsl,total_fighter_hsa,total_fighter_hsl,total_fighter_kd,total_fighter_lsa,total_fighter_lsl,total_fighter_passes,total_fighter_reverses,total_fighter_ssa,total_fighter_ssl,total_fighter_sub_att,total_fighter_tda,total_fighter_tdl,total_fighter_tsa,total_fighter_tsl
0,6,4,5,5,36,17,17,9,47,22,1,5,5,1,0,58,31,1,1,1,62,35,10,7,1,0,64,29,0,0,47,16,0,8,6,0,0,65,29,0,2,2,66,30,16,8,3,1,106,43,0,0,86,29,0,7,7,0,0,109,44,0,2,1,111,46,f9b200db02b488d9,NaN,D,32,19,9,6,206,89,17,9,180,67,1,20,18,1,0,232,104,1,5,4,239,111
0,9,7,7,7,41,16,0,0,31,10,0,8,6,0,0,48,23,0,2,1,50,24,7,4,11,8,66,24,0,0,66,24,0,4,4,0,0,77,32,0,1,0,84,38,6,4,4,4,103,34,0,0,98,32,0,3,2,0,0,107,38,0,0,0,112,42,NaN,887961364be5ceb3,D,22,15,22,19,210,74,0,0,195,66,0,15,12,0,0,232,93,0,3,1,246,104
